## Polytope Extremes-DT Feature Extraction Timeseries example notebook

This notebook shows how to use earthkit-data and earthkit-plots to pull destination-earth data from LUMI and plot it using earthkit-plots.


Before running the notebook you need to set up your credentials. See the main readme of this repository for different ways to do this or use the following cells to authenticate.

You will need to generate your credentials using the desp-authentication.py script.

This can be run as follows:

In [ ]:
!python3 ~/polytope-examples/desp-authentication.py -u <user_name> -p <password>

This will generate a token that can then be used by earthkit and polytope.

You can also run the script without arguments to enter your username and password.

In [ ]:
!python3 ~/polytope-examples/desp-authentication.py 

# Requirements
To run this notebook install the following:
* pip install earthkit-data
* pip install earthkit-regrid  (Optional for spectral variables)
* pip install cf-units         (Optional for unit conversion in maps)

In [ ]:
import earthkit.data
import earthkit.regrid

LOCATION = ((-9.11, 38.79))

request = {
    "dataset": "extremes-dt",
    "class": "d1",
    "stream": "oper",
    "type": "fc",
    "date": "20250103",
    "time": "0000",
    "levtype": "sfc",
    "expver": "0001",
    "param": "165/167",
    "step": "1/2/3/4/5/6/7/8/9",
    "feature" : {
        "type" : "timeseries",
        "points": [[LOCATION[0], LOCATION[1]]],
        "axes": "step",
    },
}

#data is an earthkit streaming object but with stream=False will download data immediately 
ds = earthkit.data.from_source("polytope", "polytope", request, stream=False, address='polytope-test.lumi.apps.dte.destination-earth.eu')


In [ ]:
ds._json()

In [48]:
# Convert data to xarray
ds.to_xarray()

<xarray.Dataset> Size: 328B
Dimensions:   (x: 1, y: 1, z: 1, number: 1, datetime: 1, t: 9)
Coordinates:
  * x         (x) float64 8B -9.122
  * y         (y) float64 8B 38.77
  * z         (z) int64 8B 0
  * number    (number) int64 8B 0
  * datetime  (datetime) <U20 80B '2025-01-03T00:00:00Z'
  * t         (t) datetime64[ns] 72B 2025-01-03T01:00:00 ... 2025-01-03T09:00:00
Data variables:
    10u       (x, y, z, number, datetime, t) float64 72B -0.6947 ... -1.132
    2t        (x, y, z, number, datetime, t) float64 72B 296.3 296.1 ... 303.8
Attributes:
    class:          d1
    dataset:        extremes-dt
    Forecast date:  2025-01-03T00:00:00Z
    expver:         0001
    levtype:        sfc
    stream:         oper
    type:           fc
    number:         0

In [49]:
def location_to_string(location):
    """
    Converts latitude and longitude to a string representation with degrees
    and N/S/E/W.
    """
    (lat, lon) = location
    lat_dir = "N" if lat >= 0 else "S"
    lon_dir = "E" if lon >= 0 else "W"
    return f"{abs(lat):.2f}°{lat_dir}, {abs(lon):.2f}°{lon_dir}"

In [50]:
from earthkit.plots.interactive import Chart

TIME_FREQUENCY = "1h"
QUANTILES = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]

chart = Chart()
chart.title(f"ECMWF ensemble meteogram at {location_to_string(LOCATION)}")
#chart.box(ds, time_frequency=TIME_FREQUENCY, quantiles=QUANTILES)
chart.line(ds, line_color='grey', time_frequency=TIME_FREQUENCY)
chart.show()  # Replace with chart.show() in an interactive session!